<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy import stats as st
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from pandas.core.common import SettingWithCopyWarning
from sklearn.metrics import r2_score

В первую очередь загрузим наши данные, проверим наличие пропусков

In [ ]:
alpha = pd.read_csv('/datasets/insurance.csv') 

In [ ]:
alpha.head()

Пол  Возраст  Зарплата  Члены семьи  Страховые выплаты
0    1     41.0   49600.0            1                  0
1    0     46.0   38000.0            1                  1
2    0     29.0   21000.0            0                  0
3    0     21.0   41700.0            2                  0
4    1     28.0   26100.0            0                  0

In [ ]:
alpha.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
alpha.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Переименуем столбцы на английский язык

In [ ]:
alpha = alpha.rename(columns={'Пол': 'Sex', 'Возраст':'Age',
                              'Зарплата': 'Salary', 'Члены семьи': 'Family', 
                              'Страховые выплаты': 'Count'})

In [ ]:
alpha.head()

Sex   Age   Salary  Family  Count
0    1  41.0  49600.0       1      0
1    0  46.0  38000.0       1      1
2    0  29.0  21000.0       0      0
3    0  21.0  41700.0       2      0
4    1  28.0  26100.0       0      0

Пропуски не обнаружены. Для столбцов Возраст и Зарплата используем pd.to_numeric() для преобразования аргумента в числовой тип.

In [ ]:
alpha['Age'] = pd.to_numeric(alpha['Age'])
alpha['Salary'] = pd.to_numeric(alpha['Salary'])

## Умножение матриц

Выведем параметры и целевой признак, напишем формулу для модели и поиска метрики r2

In [ ]:
features = alpha.drop('Count',axis=1)
target = alpha['Count']

In [ ]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = (np.linalg.inv(X.T.dot(X))).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0
    
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))


0.42494550286668


Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Значение метрики r2 не изменится

**Обоснование:** 
Заменим нашу матрицу X на рандомную R. Новая матрица - это умножение X на матрицу D. Тогда: 
$$
𝑎1 = R(R^T R)^{-1} R^Ty
$$

$$
𝑎1 = RR^{-1}(R^T)^{-1} R^Ty
$$
 


При умножении обратной матрицы $A^{-1}$ на матрицу $A$ получается единичная матрица $E$ (квадратная матрица, на главной диагонали которой стоят единицы, а остальные элементы — нули). Произведение любой матрицы и единичной матрицы подходящего размера равно самой матрице:
$$
(A^𝑇)^{-1}A^𝑇 = E
$$

Для начала вспомним, что $$(A*B)^T = A^T * B^T$$

Немного изменим формулу, оставим Х матрицу, и умножим ее на радномную обратимую матрицу O

$$
w = ((XO)^T(XO))^{-1}(XO)^Ty
$$

Начнем раскрывать скобки. Начнем с последней

$$
w = ((XO)^T(XO))^{-1}X^TO^Ty
$$

Раскроем первую общую скобку
    
$$
w = (O^TX^T(XO))^{-1}X^TO^Ty
$$

$$
w = O^{-1}(X^TX)^{-1}(O^T)^{-1}O^TX^Ty
$$

При умножении обратной матрицы  𝐴−1  на матрицу  𝐴  получается единичная матрица  𝐸  (квадратная матрица, на главной диагонали которой стоят единицы, а остальные элементы — нули). Произведение любой матрицы и единичной матрицы подходящего размера равно самой матрице:

$$
(A^𝑇)^{-1}A^𝑇 = E
$$

Сократим квадратные матрицы

$$
w = O^{-1}(X^TX)^{-1}X^T
$$

$$
a1 = (OX)w
$$

$$
a1 = (OX)O^{-1}(X^TX)^{-1}X^T 
$$

$$
a1 = X(X^TX)^{-1}X^T
$$

$$
a1 = a
$$

## Алгоритм преобразования

**Алгоритм**
Для проверки нашей теории будем умножать матрицу признаков на обратимую матрицу $RANDOM$, которая будет генерироваться случайным образом. Воспользуемся функциями np.random.rand для создания матрицы 4х4 (такое количесвто нужно для корректого умножения на наши признаки) и np.linalg.inv для создания обратной матрицы

In [ ]:
RANDOM = np.random.rand(4,4)
RANDOM2 = np.linalg.inv(RANDOM)

In [ ]:
RANDOM

array([[0.42087074, 0.64040377, 0.96837636, 0.98154839],
       [0.87682786, 0.00954652, 0.48116937, 0.0876791 ],
       [0.42857535, 0.48881621, 0.07987755, 0.60891481],
       [0.81548394, 0.00534382, 0.80400102, 0.82210829]])

In [ ]:
RANDOM2

array([[-0.60226622,  1.01531393,  0.76875454,  0.04138822],
       [ 1.03643304,  0.87878666,  0.69090002, -1.8428955 ],
       [ 1.17949982,  0.48090575, -1.55127337, -0.310553  ],
       [-0.56284452, -1.48315864,  0.7500549 ,  1.49102208]])

In [ ]:
RANDOM @ RANDOM2

array([[ 1.00000000e+00,  9.54424657e-17, -1.08723395e-16,
        -1.03893922e-16],
       [ 4.71106503e-17,  1.00000000e+00,  2.66723574e-17,
        -9.79489704e-20],
       [-2.36544093e-16,  2.71195654e-17,  1.00000000e+00,
        -1.92840409e-17],
       [ 2.59858394e-17,  1.72655586e-16, -1.96331089e-17,
         1.00000000e+00]])

In [ ]:
import numpy as np

nrows = 4

ncols = 4

a = np.random.randint(100, size=(nrows,ncols))

a

array([[81,  4, 36, 51],
       [75, 82, 73, 12],
       [33, 98, 74, 75],
       [22, 19, 49, 47]])

In [ ]:
a2 = np.linalg.inv(a)

In [ ]:
b  = a @ a2

In [ ]:
b 

array([[ 1.00000000e+00, -3.46944695e-18,  4.85722573e-17,
         1.21430643e-17],
       [ 3.88578059e-16,  1.00000000e+00,  2.22044605e-16,
        -1.38777878e-16],
       [ 5.55111512e-17,  7.97972799e-17,  1.00000000e+00,
        -2.94902991e-17],
       [ 0.00000000e+00,  3.81639165e-17, -9.02056208e-17,
         1.00000000e+00]])

In [ ]:
X_inv = features @ RANDOM2

In [ ]:
   
model = LinearRegression()
model.fit(X_inv, target)
predictions = model.predict(X_inv)
print(r2_score(target, predictions))


0.42494550286668153


## Проверка алгоритма

Для проверки случайныйм образом умножим наши параметры добавим рандомное число и домножим на созданную матрицу

In [ ]:
features_new = ((features*25 + 1889) * 15) @ RANDOM2

In [ ]:
features_new.head(10)

0             1             2             3
0  2.198397e+07  8.983453e+06 -2.882384e+07 -5.821643e+06
1  1.685531e+07  6.892780e+06 -2.207479e+07 -4.474208e+06
2  9.329607e+06  3.821960e+06 -1.219011e+07 -2.483243e+06
3  1.848194e+07  7.551242e+06 -2.423338e+07 -4.887264e+06
4  1.158479e+07  4.741744e+06 -1.515689e+07 -3.076469e+06
5  1.818065e+07  7.432635e+06 -2.382018e+07 -4.820932e+06
6  1.760409e+07  7.196876e+06 -2.306497e+07 -4.666773e+06
7  1.711168e+07  6.992776e+06 -2.242813e+07 -4.527877e+06
8  2.202626e+07  8.999840e+06 -2.888330e+07 -5.829833e+06
9  2.290933e+07  9.359201e+06 -3.004780e+07 -6.059983e+06

Значения в наших признаках теперь заполнены совершенно иныи числами, проверим изменится ли значение метрики r2

In [ ]:
 
model = LinearRegression()
model.fit(features_new, target)
predictions = model.predict(features_new)
print(r2_score(target, predictions))

0.42494550286649724


## Вывод

Мы загрузили данные, проверили на наличие пропусков столбцы. Для корректного использования поменяли названия на английский язык. Из таблицы вывели таргет и параметры. После этого написали функцию для обучения нашей модели. После этого на примере формулы доказали теорию, что при умножении метрик на матрицу результат r2 не должен поменяться. После этого мы провели исследования нескольких обратимых матриц, проверили на практике теорию о том, что если перемножить матрицу с обратной матрицей, то получается единичная матрица. После этого мы количественно изменили наши метрики для проверки гипотезы на практике. В результате значение r2 не поменялось. 

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования